<a href="https://colab.research.google.com/github/kaigouthro/InvokeAI_EasyBook/blob/main/InvokeAI_EasyBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check if you have a gpu..
!nvidia-smi -L
!nvidia-smi

## Sign up/accept terms/get tokens at 
-  https://huggingface.co/runwayml/stable-diffusion-v1-5 
-  https://huggingface.co/runwayml/stable-diffusion-inpainting 
-  https://ngrok.com/  (for access outside)


In [ ]:
#@title Input: { vertical-output: true, form-width: "20%", display-mode: "form" }
from google.colab import drive
import os
import sys

# Set if instance should be temporary or persistent
persist_in_drive            = True #@param {type:"boolean"}

# Set Path either way, it can go temp or to drive.
workspace  = '/content/drive/MyDrive/invoke_ai'

# Set up storage
if persist_in_drive:
  # mount if chosen
    drive.mount('/content/drive')

# create path if not exists (persistent or temp, ssame folder)
os.makedirs(workspace,exist_ok=True)

# Sign up/accept terms/get tokens at https://huggingface.co/runwayml/stable-diffusion-v1-5 https://huggingface.co/runwayml/stable-diffusion-inpainting https://ngrok.com/
huggingface_token = "" #@param {type:"string"}
ngrok_token = "" #@param {type:"string"}

----
# **` Prep -  you can RUN all the rest from here`**
----

In [ ]:
#@title #install the dependencies.
# Install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y
!pip install pyngrok --quiet
!pip3 install virtualenv 

# Change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

# Verify python version
!python --version

# Update pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall
!python -m pip install --upgrade pip
!pip install --upgrade setuptools

# Handle colab dependencies
!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

In [ ]:
#@title Create InvokeAI root
# Create InvokeAI root
invokeai_root  = workspace + '/invokeai'
os.makedirs ( invokeai_root  , exist_ok = True )
os.chdir    ( invokeai_root  )
print       ( os.getcwd())     


In [ ]:
#@title  Create and prepare virtual environment
!virtualenv venv --system-site-packages
!source venv/bin/activate #activate the virtual env

# `___________` Install `Invoke` and run `____________`

In [ ]:
#@title # ` install InvokeAI `
!source venv/bin/activate; pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
#@title #### `(skip the login prompt this way for faster running)`
import os
!source venv/bin/activate;  os.environ['HUGGING_FACE_HUB_TOKEN'] = huggingface_token
!source venv/bin/activate;  os.environ  ['INVOKEAI_ROOT'] = invokeai_root
!source venv/bin/activate; invokeai-configure --yes
!sed -i 's/--nsfw_checker/--no-nsfw_checker/' invokeai.init

!source venv/bin/activate; pip install huggingface_hub  "ipywidgets>=7,<8" --upgrade
!source venv/bin/activate; huggingface-cli login --token=$huggingface_token

In [ ]:
#@title # `NGROK proxy and launch Invoke`
from pyngrok import ngrok
from IPython.display import display, Markdown
ngrok.kill()
!ngrok.set_auth_token($ngrok_token)
public_url = ngrok.connect(9090).public_url


# show the link
display(Markdown("# WEBUI \n------\n`InvokeAI Public URL :`\n"))
print(f'{public_url}')
display(Markdown("\n\n\n------\n\n ` click once the next cell is finshed"))


In [ ]:
#@title # ` start it up `
os.chdir(invokeai_root)
!source venv/bin/activate; invokeai --no-nsfw_checker --no-safety_checker --web  --xformers --host 0.0.0.0